# Homework 5 - Visit the Wikipedia hyperlinks graph!

# Libraries

In [1]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import pickle
import collections
import math
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import statistics
import operator
import random

### Researh question 1

[RQ1] Build the graph [G=(V, E)] , where V is the set of articles and E the hyperlinks among them, and provide its basic information:

* If it is direct or not
* The number of nodes
* The number of edges
* The average node degree. Is the graph dense?

# Data

In [2]:
# Reading reduced edges file as dataframe, where one column is the source and other is the destination
source_destination=pd.read_csv('wiki-topcats-reduced.txt',sep="\t",header=None,names=["source","destination"])

>reading of a dicionary created in _Homework 5 - RQ1 pre-check up.ipynb_ notebook file
source_destination_dict is a **default dictionary** with list as datatype for value of dictionary, where **key**=source_id(article_id), **value**=destination_id(list of article_id's which are connected to that article)

reading pickle file into a memory, which contains a dictionary where **key=source_id(article_id)**, **value=destination_id(list of article_id's** which are connected to that article)

In [3]:
source_destination_dict = pickle.load(open('source_destination_dict.p', 'rb'))

In [4]:
#list of (source, destination) tuples which will be used to read edges from them in networkx _add_edges_from_ method
source_destination_tuples=[(row["source"],row["destination"]) for idx,row in tqdm(source_destination.iterrows())]

2645247it [02:21, 18686.07it/s]


### The number of nodes and edges

Based on the results and the conclusion from the _Homework 5 - RQ1 pre-check up.ipynb_ we can use DiGraph() method to make a directed graph immediately. And check again the subquestions of the RQ1 in order to give the final response and conclusions.

In [5]:
Gtup_directed = nx.DiGraph()
Gtup_directed.add_edges_from(source_destination_tuples)
print("Number of nodes:",len(Gtup_directed.nodes()),"\nNumber of edges:",len(Gtup_directed.edges()))

Number of nodes: 461193 
Number of edges: 2645247


### Average node degree

In [6]:
print(nx.info(Gtup_directed))

Name: 
Type: DiGraph
Number of nodes: 461193
Number of edges: 2645247
Average in degree:   5.7357
Average out degree:   5.7357


###  Is the graph dense?

In [7]:
print('Density of the graph:',nx.density(Gtup_directed))

Density of the graph: 1.2436602635647606e-05


In [8]:
#The graph is a sparse graph. Because the density has a small value and it is closer to 0.

Results of the RQ1:
    
>The graph is **DIRECTED**. That is why we used DiGraph() method from nx library
 
>The **number of nodes**: 461 193

>The **number of edges**: 2 645 247

>The **average node degree** is: 5.7357

>Is the graph dense? Density = 1.2437e-05 The density can have value from 0 to 1, whereby it is 0 for a graph without edges and 1 for a complete graph. Therefore we can conclude that **the graph is sparse** and not dense.

### Researh question 2

1. Building Block Ranking

    > Based on the implementation of the **shortest path** algorithm compare sample number of nodes of C0-input category with all nodes in all the other Ci categories in order to build the **block ranking**. 

2. Ranking nodes of each category in the created block ranking vector and selecting top 3 and finding article names for it

In [9]:
#categories file, where each category has a list of articles that belongs to it
categories=pd.read_csv('wiki-topcats-categories.txt',sep="\n",header=None)

In [10]:
def num_cat(row):
   
    """
    method which performs cleaning of each row from the categories file and counting the actual number of articles that belong to that category
  
    returns the number of articles 
    """
    return len(row.split("; ")[1].split(" "))

In [11]:
#applying the lambda and num_cat method for labeling and selecting categories who have more than 3500 articles
categories=categories[categories.iloc[:,0].apply(lambda x:True if num_cat(x)>3500 else False)]

In [12]:
#cleaning the dataframe with categories and mapping the articles to integers.
#and making the categories_dict where key=category name, value=list of article id's belonging to that category
categories_dict=defaultdict(list)
for idx in categories.index:
    cat_and_values=categories[0].loc[idx].split("; ")
    cat_name=cat_and_values[0].split(":")[1]
    categories_dict[cat_name]=list(map(int,cat_and_values[1].split(" ")))

In [13]:
# We should consider as nodes, only the ones given in the reduced version of the graph. Therefore we are checking and removing
#the other ones and based on that 6 more categories are eliminated. Therefore, now we have 29 categories.
categories = {}
all_nodes=set(Gtup_directed.nodes())
for key, values in categories_dict.items():
        categories[key] = all_nodes.intersection(set(values))
        if len(categories[key]) < 3500:
            del(categories[key])

We decided to see how many edges every category has as part of the main directed graph and to choose the one that has the biggest number of edges so we can take a random sample of nodes in order to sort nodes in those categories in an easier way. 

In [14]:
cat_edges={}
for cat in categories:
    cat_edges[cat]=len(Gtup_directed.subgraph(categories[cat]).edges)We decided  to see how many edges every category has as part of the main directed graph and to choose the one that has the biggest number of edges so we can take a random sample of nodes in order to sort nodes in those categories in an easier way. 

In [15]:
cat_edges

{'English_footballers': 18233,
 'The_Football_League_players': 19672,
 'Association_football_forwards': 4710,
 'Association_football_goalkeepers': 8209,
 'Association_football_midfielders': 3801,
 'Association_football_defenders': 2286,
 'Living_people': 1218406,
 'Harvard_University_alumni': 3959,
 'Major_League_Baseball_pitchers': 10472,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 30039,
 'Indian_films': 3323,
 'Year_of_death_missing': 1242,
 'Year_of_birth_missing_(living_people)': 6814,
 'Rivers_of_Romania': 15241,
 'Main_Belt_asteroids': 10891,
 'Asteroids_named_for_people': 142,
 'English-language_albums': 7064,
 'British_films': 2805,
 'English-language_films': 21278,
 'American_films': 10372,
 'People_from_New_York_City': 3156,
 'American_television_actors': 32231,
 'American_film_actors': 53164,
 'Debut_albums': 779,
 'Black-and-white_films': 6288,
 'Year_of_birth_missing': 1112,
 'Place_of_birth_missing_(living_people)': 736,
 'American_military_pe

In [16]:
#sort descending to see category with most edges in its subgraph
sorted(cat_edges.items(), key=operator.itemgetter(0), reverse= False)

[('American_film_actors', 53164),
 ('American_films', 10372),
 ('American_military_personnel_of_World_War_II', 5133),
 ('American_television_actors', 32231),
 ('Association_football_defenders', 2286),
 ('Association_football_forwards', 4710),
 ('Association_football_goalkeepers', 8209),
 ('Association_football_midfielders', 3801),
 ('Asteroids_named_for_people', 142),
 ('Black-and-white_films', 6288),
 ('British_films', 2805),
 ('Debut_albums', 779),
 ('English-language_albums', 7064),
 ('English-language_films', 21278),
 ('English_footballers', 18233),
 ('Harvard_University_alumni', 3959),
 ('Indian_films', 3323),
 ('Living_people', 1218406),
 ('Main_Belt_asteroids', 10891),
 ('Major_League_Baseball_pitchers', 10472),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  30039),
 ('People_from_New_York_City', 3156),
 ('Place_of_birth_missing_(living_people)', 736),
 ('Rivers_of_Romania', 15241),
 ('The_Football_League_players', 19672),
 ('Windows_games', 6117),
 (

>In order to test and make a runnable version of our code, we decided to test it on the sample. We have chosen the **'American_film_actors'** as our **input category** and we will perform random sample, which is ok because it is a really sparse graph, and we decided to select **100 nodes** from the input category and **all the other nodes** from the rest of the categories.

In [18]:
#random sampling of 100 nodes from the input category 'American_film_actors' and making a sample subgraph for it
# we repeat the process until at least 10 edges exist between sampled 100 nodes.
for i in range(100):
    cat_nodes_lst = random.sample(categories['American_film_actors'], 100)
    Gtup_directed_sample_C0=Gtup_directed.subgraph(cat_nodes_lst)
    if len(Gtup_directed_sample_C0.subgraph(categories['American_film_actors']).edges)>10:
        break

In [19]:
#number of edges in whole subgraph for American_film_actors category
len(Gtup_directed.subgraph(categories['American_film_actors']).edges)

53164

In [20]:
#11 edges exist
nx.info(Gtup_directed_sample_C0)

'Name: \nType: SubDiGraph\nNumber of nodes: 100\nNumber of edges: 11\nAverage in degree:   0.1100\nAverage out degree:   0.1100'

In [21]:
def bfs_shortest_path(graph, start):
    """
    method which returns shortest paths dictionary where key=node, and value is the actual distance of the shortest path from the start node to the other nodes of the graph
    """
    explored = set()
    queue = [start]
    shortest_path_dict = {}
    counter_level = 0
    to_visit_list = []
    while queue:
        for node in queue:
            if node not in explored:
                shortest_path_dict[node] = counter_level
                neighbours = graph[node]
                explored.add(node)
                to_visit_list.extend(neighbours)
        queue = []
        queue.extend(to_visit_list)
        to_visit_list = []
        counter_level += 1
    return shortest_path_dict

In [22]:
#calculate shortest path from each article of the SubGraph of the input category sample to all the other nodes in the Graph
l = []
for article in tqdm(Gtup_directed_sample_C0.nodes()):
    l.append(bfs_shortest_path(Gtup_directed, article))   

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.09s/it]


Since we made sample of 100 nodes from the input category now we have to remove other articles in the category **['American_film_actors'].**

In order to answer to the RQ2, we performed the following steps:

1. Making of a dictionary **d** to store value of shortest paths, where key=node, value=list of shortest paths distances
2. Making of a dictionary **z** to store value of **MINIMUM shortest path**, where key=node, value=minimum of the shortest path
3. Making of a **categories_edited**, whose category **'American_film_actors'** will be consisted only of the **sampled articles**.
4. Making of a **cat_inv_dic** where key=name of the category, value=shortest path values

In [23]:
#d-dictionary, key=node, value=list of distances of all the possible shortest paths 
d = defaultdict(list)
for node in l:
    for key, value in node.items():
        d[key].append(value)

In [24]:
#z-dictionary, key=node, value=minimum distance of all the possible shortest paths 
z = defaultdict(list)
for key, value in d.items():
    z[key] = np.min(d[key])

In [25]:
#categories_edited will contain everything as categories except for the input category, which will contain only sampled nodes
categories_edited = categories.copy()

In [26]:
#to_delete_article list of articles which are not sampled, therefore not considered in testing and conclusion 
to_delete_article = []
for article in categories_edited['American_film_actors']:
    if article not in cat_nodes_lst:
        to_delete_article.append(article)

In [27]:
#deletion of other not sampled articles
for article in to_delete_article:
    categories_edited['American_film_actors'].remove(article)

As we can see now 'American_film_actors' has **100 articles** the one that we sampled from the beginning.

In [28]:
len(categories_edited['American_film_actors'])

100

In [29]:
#29 categories remained after elimination of ones that have less than 3500 articles
len(categories_edited.keys())

29

In [30]:
#cat_inv_dic, dictionary - key=name of the category, value= list of minimum shortest path values for its nodes(articles)
cat_inv_dic=defaultdict(list)
for cat_k,cat_v in tqdm(categories_edited.items()):
    for node,val in z.items():
        if node in cat_v:
            cat_inv_dic[cat_k].append(val)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 15.15it/s]


In [31]:
#checking whether the number of sampled nodes in now right-100 as should be
len(cat_inv_dic['American_film_actors'])

100

We didn't consider the calculation of the shortest paths when 2 nodes are not connected, where the 'shortest path' would have been infinite. Since infinite doesn't make sense for median calculation we thought we could add a big number, such as 100.
So where the edges between 2 nodes don't exist we extend it to values of 100 so it is considered in the median calculation.
Of course, we checked how many inf values should be based on the number of articles that should belong to that category.

In [32]:
#add value of 100 where nodes are not connected
for cat_k in categories_edited.keys():
    cat_inv_dic[cat_k].extend([100]*(len(categories_edited[cat_k])-len(cat_inv_dic[cat_k])))

In [33]:
#calculating the median
for key, value in cat_inv_dic.items():
    cat_inv_dic[key] = statistics.median(cat_inv_dic[key])

In [34]:
#cat_inv_dic, where key=name of the category, value is the calculated median
cat_inv_dic

defaultdict(list,
            {'English_footballers': 6.0,
             'The_Football_League_players': 6.0,
             'Association_football_forwards': 7,
             'Association_football_goalkeepers': 100,
             'Association_football_midfielders': 100,
             'Association_football_defenders': 11.0,
             'Living_people': 5.0,
             'Harvard_University_alumni': 5,
             'Major_League_Baseball_pitchers': 5.0,
             'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 5,
             'Indian_films': 4.0,
             'Year_of_death_missing': 100.0,
             'Year_of_birth_missing_(living_people)': 5.0,
             'Rivers_of_Romania': 6,
             'Main_Belt_asteroids': 100.0,
             'Asteroids_named_for_people': 100,
             'English-language_albums': 4.0,
             'British_films': 3.0,
             'English-language_films': 3,
             'American_films': 3,
             'People_from_New_York_City':

In [35]:
#sort descending
cat_inv_dic_rank= sorted(cat_inv_dic.items(), key=operator.itemgetter(1), reverse= False)

In [36]:
#Block ranking vector
cat_inv_dic_rank

[('American_film_actors', 0.0),
 ('British_films', 3.0),
 ('English-language_films', 3),
 ('American_films', 3),
 ('American_television_actors', 3),
 ('Black-and-white_films', 3),
 ('Indian_films', 4.0),
 ('English-language_albums', 4.0),
 ('People_from_New_York_City', 4.0),
 ('Debut_albums', 4),
 ('Living_people', 5.0),
 ('Harvard_University_alumni', 5),
 ('Major_League_Baseball_pitchers', 5.0),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 5),
 ('Year_of_birth_missing_(living_people)', 5.0),
 ('Place_of_birth_missing_(living_people)', 5.0),
 ('American_military_personnel_of_World_War_II', 5.0),
 ('Windows_games', 5),
 ('English_footballers', 6.0),
 ('The_Football_League_players', 6.0),
 ('Rivers_of_Romania', 6),
 ('Association_football_forwards', 7),
 ('Association_football_defenders', 11.0),
 ('Association_football_goalkeepers', 100),
 ('Association_football_midfielders', 100),
 ('Year_of_death_missing', 100.0),
 ('Main_Belt_asteroids', 100.0),
 ('Asteroid

In [37]:
columns = ['Category name', 'Median']

block_ranking_df = pd.DataFrame([x for x in cat_inv_dic_rank], columns=columns)

In [38]:
block_ranking_df.index.names = ['Ranking']

In [39]:
block_ranking_df

,Category name,Median
Ranking,,
0,American_film_actors,0.0
1,British_films,3.0
2,English-language_films,3.0
3,American_films,3.0
4,American_television_actors,3.0
5,Black-and-white_films,3.0
6,Indian_films,4.0
7,English-language_albums,4.0
8,People_from_New_York_City,4.0


As we can see 'American_film_actors' is our input category and it has median 0 which is logical cause it is an input category and shortest distances are 0 between that category and its nodes.

In [41]:
#Reading the names of the articles from the file

In [42]:
#article_names_dict, dictionary, where key=article id, value=article name
article_names_dict={}
with open('wiki-topcats-page-names.txt') as file:
    for line in file:
        id_name=line.rstrip().split(' ')
        article_names_dict[int(id_name[0])]=" ".join(id_name[1:])

In [43]:
#example of first article
article_names_dict[0]

'Chiasmal syndrome'

In [44]:
#list of ranked categories
cat_lst=list(block_ranking_df['Category name'])

Computation of the subgraph induced by input category-C0. For each node the sum of **the weigths of the in-edges** should be computed.

In [45]:
input_category=cat_lst[0]

sub_graph_Cat=Gtup_directed.subgraph((list(categories_edited[input_category])))#make a subgraph containing just THAT category
in_edge_All=[] #list of first 3 sorted nodes for each category 
dict_previous_cat={} #dictionary containing a previously ranked category of the current category,key=node, value=number of in-edges 

#just for first category-input category C0
in_edge_temp_dict = defaultdict(int) #temporary dictionary to store as key=node, value=number of in-edges 
for tupla in list(sub_graph_Cat.edges()): # tupla-edge tuple, (source node, destination node)
    if tupla[1] in categories_dict[input_category]: # if the destination node belongs to the current C0 category
        in_edge_temp_dict[tupla[1]] += 1 #add one to that destination node in order to count all the in-edges

sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True) # sort the nodes in that category
in_edge_All.append(sorted_in_edge_dict[0:3]) #add sorted dictionary of nodes to a list 

In [46]:
#top 3 sorted nodes from the first input category
in_edge_All

[[(1163611, 2), (1062014, 2), (1163947, 1)]]

In [47]:
#number of edges in the sampled input category
print(len(list(sub_graph_Cat.edges())))

11


In [49]:
in_edge_dict=in_edge_temp_dict #take nodes and values from the 1st input category
for cat in tqdm(cat_lst[1:]):#for other 28 categories
    in_edge_temp_dict = defaultdict(int)
    sub_graph_Cat=Gtup_directed.subgraph((list(categories_edited[cat])))#make a subgraph containing just THAT category
    for tupla in list(sub_graph_Cat.edges()): # go to every node of the cat subgraph
        try:
            source=in_edge_dict[tupla[0]] #take the source node from the previous category
            in_edge_temp_dict[tupla[1]] += source #add in-edge value from the previous node
        except:
            in_edge_temp_dict[tupla[1]] += 1 #append 1 if it is not there
    in_edge_dict.update(in_edge_temp_dict)

    sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True)
    in_edge_All.append(sorted_in_edge_dict[0:3])

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.01it/s]


In [50]:
len(in_edge_All)

29

In [51]:
in_edge_All

[[(1163611, 2), (1062014, 2), (1163947, 1)],
 [(1056770, 0), (1056769, 0), (1041937, 0)],
 [(1062743, 0), (1064184, 0), (1064265, 0)],
 [(1245946, 0), (688149, 0), (688300, 0)],
 [(1163623, 2), (1061808, 2), (1061750, 2)],
 [(589847, 0), (590961, 0), (1063431, 0)],
 [(582698, 0), (590800, 0), (589828, 0)],
 [(1228798, 0), (1228799, 0), (145526, 0)],
 [(1061452, 2), (1023762, 2), (1164816, 2)],
 [(566701, 0), (328881, 0), (155018, 0)],
 [(1062014, 12), (1061284, 8), (1061865, 8)],
 [(1062012, 4), (1380119, 4), (1400478, 2)],
 [(386079, 0), (384805, 0), (387201, 0)],
 [(543595, 1), (540950, 1), (537478, 1)],
 [(643206, 0), (973046, 0), (655275, 0)],
 [(1151940, 0), (491635, 0), (1562881, 0)],
 [(1164934, 8), (1061441, 8), (1025555, 8)],
 [(1734268, 0), (1507349, 0), (1735504, 0)],
 [(80379, 0), (87391, 0), (78944, 0)],
 [(81787, 0), (81941, 0), (82484, 0)],
 [(785340, 0), (786385, 0), (786433, 0)],
 [(1358244, 0), (88262, 0), (884966, 0)],
 [(79221, 0), (81524, 0), (81919, 0)],
 [(737293

In [54]:
#print the block ranking vector and top 3 articles which belong to each of the category
for idx,row in enumerate(in_edge_All):
    row_lst=[]
    for tup in row:
        tuple_new=(article_names_dict[tup[0]],tup[1])
        row_lst.append(tuple_new)
    print(idx,cat_lst[idx],row_lst)

0 American_film_actors [('John Payne (actor)', 2), ('Christian Slater', 2), ('Meredith Baxter', 1)]
1 British_films [('The Golden Voyage of Sinbad', 0), ('Sinbad and the Eye of the Tiger', 0), ('Cinema of the United Kingdom', 0)]
2 English-language_films [('Reservoir Dogs', 0), ('Predators (film)', 0), ('Machete (film)', 0)]
3 American_films [('Educating Peter', 0), ('Behind the Green Door', 0), ('Deep Throat (film)', 0)]
4 American_television_actors [('George Burns', 2), ('Emilio Estevez', 2), ('Michael J. Fox', 2)]
5 Black-and-white_films [('Ladki', 0), ('Penn (film)', 0), ('Duck Soup (1933 film)', 0)]
6 Indian_films [('Shola Aur Shabnam (1992 film)', 0), ('Illarikam', 0), ('Aasha (1957 film)', 0)]
7 English-language_albums [('Bloodrock U.S.A.', 0), ('Bloodrock 2', 0), ('Wild in the Streets (Circle Jerks album)', 0)]
8 People_from_New_York_City [('Lauren Bacall', 2), ('Artie Shaw', 2), ('William S. Paley', 2)]
9 Debut_albums [('America (America album)', 0), ('No One Can Do It Better'

In our input category **American_film_actors** based on the random sample the most popular article is about **'Elvis Presley'**, who played in 31 movies as an actor ;)